# Cell Below is CSV Combiner 

Run the cell below using the triangle button on the top left or clicking in the cell and pressing Shift + Enter to run it. The code will use the files names from the "data" folder. It will assume protein names from each file names by using everything before the first underscore (_) then will combine into one csv file with columns as different proteins from each file. 

Note that there will be a preview of the first 10 rows of what the final csv looks like so you can check it before moving to save it in the following cell. 

In [17]:
import pandas as pd
from IPython.display import FileLink
import os

def merge_protein_csvs(folder_path):
    # Step 1: Get all CSVs
    csv_files = [f for f in os.listdir(folder_path) if f.endswith('.csv')]

    # Step 2: Identify the DAPI file (case-insensitive search)
    dapi_file = next((f for f in csv_files if 'dapi' in f.lower()), None)
    if not dapi_file:
        raise ValueError("No DAPI file found in folder.")

    # Helper function to clean and standardize column names
    def clean_and_standardize_columns(df):
        df.columns = [col.strip().rstrip(',').upper() for col in df.columns]
        col_map = {}
        for col in df.columns:
            if 'POSITION' in col and 'X' in col:
                col_map[col] = 'POSITION X'
            elif 'POSITION' in col and 'Y' in col:
                col_map[col] = 'POSITION Y'
        return df.rename(columns=col_map)

    # Step 3: Load DAPI DataFrame as base
    dapi_df = pd.read_csv(os.path.join(folder_path, dapi_file), skiprows=3)
    dapi_df = clean_and_standardize_columns(dapi_df)

    # Ensure we don't modify DAPI filename in the loop
    remaining_files = [f for f in csv_files if f != dapi_file]

    # Step 4: Process each protein file
    for file in remaining_files:
        filepath = os.path.join(folder_path, file)
        df = pd.read_csv(filepath, skiprows=3)
        df = clean_and_standardize_columns(df)

        # Drop duplicates of position coordinates
        df = df[['POSITION X', 'POSITION Y']].drop_duplicates()

        # Extract protein name and convert to uppercase
        protein_name = file.split('_')[0].upper()

        # Mark presence of (X,Y) in current protein file
        df[protein_name] = 1

        # Merge into base (DAPI) on POSITION X and Y
        dapi_df = dapi_df.merge(df, on=['POSITION X', 'POSITION Y'], how='left')

    # Step 5: Replace NaN with 0 in presence columns
    presence_cols = [col for col in dapi_df.columns
                 if col not in ['POSITION X', 'POSITION Y'] and pd.api.types.is_numeric_dtype(dapi_df[col])]

# Fill NaNs and cast to int
    dapi_df[presence_cols] = dapi_df[presence_cols].fillna(0).astype(int)

    return dapi_df


folder_path = 'data'
combined_csv = merge_protein_csvs(folder_path)
combined_csv


/var/folders/bv/7wzy33ls6ln4ydwr6s8vwq8h0000gn/T/ipykernel_31100/1986682359.py:26: DtypeWarning: Columns (7,8) have mixed types. Specify dtype option on import or set low_memory=False.
  dapi_df = pd.read_csv(os.path.join(folder_path, dapi_file), skiprows=3)


,POSITION X,POSITION Y,POSITION Z,UNIT,CATEGORY,COLLECTION,TIME,SET 1,USELESS,ID,...,CD11B OFF F480,CD206 OFF F480,CD4,LY6G,F480,CD3,NK1.1,CD11B,CD8,MHCII
0,71200.969,45685.188,-22,µm,Surface,Position,1,NaN,NaN,38172,...,0,0,0,0,0,0,0,0,0,0
1,72096.891,44789.438,-19,µm,Surface,Position,1,NaN,NaN,38173,...,0,0,0,0,0,0,0,0,0,0
2,71387.711,45500.258,-20,µm,Surface,Position,1,NaN,NaN,38174,...,0,0,0,0,0,0,0,0,0,0
3,71587.172,45303.039,-24,µm,Surface,Position,1,NaN,NaN,38175,...,0,0,0,0,0,0,0,0,0,0
4,71925.766,44961.535,-19,µm,Surface,Position,1,NaN,NaN,38176,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
722318,69372.898,44787.383,-19,µm,Surface,Position,1,NaN,Class A,116161,...,0,0,0,0,0,0,0,0,0,0
722319,69422.750,44695.879,-19,µm,Surface,Position,1,NaN,Class A,126618,...,0,0,0,0,0,0,0,1,0,0
722320,69426.352,44698.328,-23,µm,Surface,Position,1,NaN,Class A,126637,...,0,0,0,0,0,0,0,1,0,0
722321,69428.562,44454.734,-20,µm,Surface,Position,1,NaN,Class A,128048,...,0,0,0,0,0,0,0,1,0,0


# Use cell below to save the combined csv file after ensuring the preview looks correct above. 

Run the cell below by clicking the triangle in the top left or clicking in the cell and pressing shift + enter. Make sure to renmae the file name to whatever you need in the part to left of the green arrow make sure to leave the name in the quotes. 

The combined csv file will be saved in a folder called "combined_csvs" there will also be a hyperlink below that will open up the csv file to make sure you can see where it is stored and view the full file. 

In [18]:
output_dir = "csv_for_tools"
os.makedirs(output_dir, exist_ok=True)  

file_name = "merged_for_tools.csv" 
output_path = os.path.join(output_dir, file_name)

combined_csv.to_csv(output_path, index=False)

FileLink(output_path)


/Users/patrick/Desktop2/lab_proj1/csv_for_tools/merged_for_tools.csv